## Random Forest Approach



### Setup

In [1]:
import pandas as pd
import imp
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

import os 
from pathlib import Path


/tmp/ipykernel_15570/4288243911.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [2]:
# read dynamic path
base_dir = Path(os.getcwd()) / "implementation"
data_dir = base_dir / "data/source/"
result_dir = base_dir / "data/results/"

### load & prepare dataset

the following code needs to be adapted for each protein-ligand complex individually

In [3]:
rf_data_raw = pd.read_csv(data_dir / "SEH/SEH.csv")

rf_data_raw

,INDEX,NAME,Hydrophobic_Interaction:PHE387A,Hydrogen_Bond:TYR466A,Hydrogen_Bond:TRP336A,Salt_Bridge:HIS524A,Hydrophobic_Interaction:ASN472A,Hydrogen_Bond:ASN472A,Water_Bridge:ALA365A,Pi-Cation_Interaction:HIS524A,...,Water_Bridge:PHE267A,Hydrophobic_Interaction:PRO371A,Hydrophobic_Interaction:LEU428A,Hydrophobic_Interaction:ALA365A,Hydrogen_Bond:LEU408A,Hydrogen_Bond:TYR383A,Hydrogen_Bond:TYR343A,Hydrophobic_Interaction:HIS524A,Halogen_Bond:ASN472A,LABEL
0,1,IA_147.cdx|SeH_inactivess_minimized|sdf|48,1,1,1,0,0,0,0,0,...,0,0,0,0,0,2,0,0,1,inactive
1,2,IA_10|SeH_inactivess_minimized|sdf|1,0,1,0,1,0,0,0,0,...,2,0,0,0,0,2,0,0,0,inactive
2,3,A_103|SEH_inhibs_minimized|sdf|5,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,active
3,4,A_76|SEH_inhibs_minimized|sdf|46,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,active
4,5,IA_12|SeH_inactivess_minimized|sdf|22,0,1,0,1,1,0,0,0,...,1,1,0,0,0,0,0,0,0,inactive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,194,IA_178.cdx|SeH_inactivess_minimized|sdf|79,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,inactive
194,195,IA_211.cdx|SeH_inactivess_minimized|sdf|109,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,inactive
195,196,IA_155.cdx|SeH_inactivess_minimized|sdf|56,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,inactive
196,197,IA_163.cdx|SeH_inactivess_minimized|sdf|65,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,inactive


In [4]:
lookup = {'inactive':0,'active':1}

rf_data = {'data': np.array(rf_data_raw.iloc[:, 2:-1]),
             'target': np.array([lookup[y] for y in rf_data_raw.iloc[0:,-1]]),
             'feature_names': rf_data_raw.columns[2:-1],
             'target_names': ['inactive', 'active']}


split into train- and test-set

In [5]:
X_train, X_test, y_train, y_test = train_test_split(rf_data['data'], rf_data['target'],
                                                    test_size=0.3, random_state=4232)

### Apply Random Forrest

In [6]:
parameters = {'max_depth': [i for i in range(3, 28,3)]}
rf_models = GridSearchCV(RandomForestClassifier(n_estimators=1001, oob_score=True), parameters, cv=20, n_jobs=-1)


Fit model with the provided parameters

In [7]:
rf_models.fit(X_train, y_train)

GridSearchCV(cv=20,
             estimator=RandomForestClassifier(n_estimators=1001,
                                              oob_score=True),
             n_jobs=-1,
             param_grid={'max_depth': [3, 6, 9, 12, 15, 18, 21, 24, 27]})

In [8]:
pd.DataFrame(rf_models.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,split13_test_score,split14_test_score,split15_test_score,split16_test_score,split17_test_score,split18_test_score,split19_test_score,mean_test_score,std_test_score,rank_test_score
0,9.384434,0.356486,0.205131,0.047563,3,{'max_depth': 3},0.857143,0.857143,0.857143,0.714286,...,0.714286,0.714286,0.714286,0.714286,0.714286,0.833333,0.833333,0.747619,0.058029,3
1,7.897578,0.677743,0.216041,0.056944,6,{'max_depth': 6},0.857143,0.714286,0.857143,0.714286,...,0.714286,0.857143,0.714286,0.714286,0.714286,0.833333,0.833333,0.754762,0.062133,2
2,5.884856,0.550924,0.133730,0.029314,9,{'max_depth': 9},0.857143,0.714286,0.857143,0.714286,...,0.714286,0.857143,0.714286,0.714286,0.714286,0.833333,0.833333,0.769048,0.067386,1
3,5.156585,0.400590,0.150768,0.037837,12,{'max_depth': 12},0.857143,0.714286,0.714286,0.714286,...,0.571429,0.714286,0.714286,0.714286,0.714286,0.833333,0.833333,0.747619,0.073540,3
4,7.800504,0.526052,0.116565,0.024485,15,{'max_depth': 15},0.857143,0.714286,0.857143,0.714286,...,0.571429,0.714286,0.714286,0.714286,0.714286,0.833333,0.833333,0.747619,0.086307,3
5,4.806604,0.286690,0.118452,0.022668,18,{'max_depth': 18},0.857143,0.714286,0.857143,0.714286,...,0.571429,0.714286,0.714286,0.714286,0.714286,0.833333,0.833333,0.747619,0.086307,3
6,4.569507,0.322139,0.112894,0.027443,21,{'max_depth': 21},0.857143,0.714286,0.857143,0.714286,...,0.571429,0.714286,0.714286,0.714286,0.714286,0.833333,0.833333,0.740476,0.082787,8
7,4.478901,0.332530,0.127684,0.037964,24,{'max_depth': 24},0.857143,0.714286,0.857143,0.714286,...,0.571429,0.714286,0.714286,0.714286,0.714286,0.833333,0.833333,0.747619,0.086307,3
8,3.952830,0.353885,0.075161,0.024629,27,{'max_depth': 27},0.857143,0.714286,0.857143,0.714286,...,0.571429,0.714286,0.714286,0.714286,0.714286,0.833333,0.833333,0.740476,0.082787,8


In [9]:
rf_models.best_score_

0.769047619047619

In [10]:
rf_models.best_params_

{'max_depth': 9}

### Score

In [11]:
prediction = rf_models.best_estimator_.predict(X_test)
rf_models.best_estimator_.score(X_test,y_test)

0.7666666666666667

### Prepare Data for evaluation

In [12]:
result_df = pd.DataFrame(columns=rf_data_raw.columns[:-1])
result_df = result_df.drop(columns="NAME")

for i, row in enumerate(X_test):
    data = [i]
    data.extend(row)
    result_df.loc[len(result_df["INDEX"])] = data

result_df["LABEL"] = y_test
result_df["PRED"] = prediction

result_df.to_csv(result_dir/"SEH/baseline_rf.csv",encoding="utf-8")